In [1]:
from kafka import KafkaProducer
import requests
from json import dumps, loads
from time import sleep
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

# Read credentials
ip_address = config.get('AWS','ip_address')
topic_name = config.get('KAFKA','topic_name')
port = config.get('KAFKA','port')
yahoo_finance_host = config.get('YAHOO_FINANCE','host')
yahoo_finance_api_key = config.get('YAHOO_FINANCE','api_key')

In [3]:
producer = KafkaProducer(
                        bootstrap_servers=f'{ip_address}:{port}',
                        value_serializer=lambda x: dumps(x).encode('utf-8'))

In [4]:
method = 'market/v2/get-quotes'
url = f'https://{yahoo_finance_host}/{method}'

headers = {
	"X-RapidAPI-Key": yahoo_finance_api_key,
	"X-RapidAPI-Host": yahoo_finance_host
}
querystring = {"region":"US"}

In [16]:
#AAPL,IBM,MSFT,AMD,INTC,GOOG,FB,TSLA,AMZN,ORCL
#ticker = str(input('Introduce ticker: ')).upper()

tickers = ['AAPL','IBM','MSFT','AMD','INTC','GOOG','FB','TSLA','AMZN','ORCL','MMM','MCD','META','SPOT','IAG','ITX',
			'RIVN','VUSA','WRD','VFEM','XDJP','MO','PEP','NKE','USB','LMT','HII','FCPT','DLR','NSA','KNOP']
for ticker in tickers:

	querystring['symbols'] = ticker

	response = requests.request("GET",
								url,
								headers=headers,
								params=querystring)

	response = loads(response.text)

	producer.send(topic_name, value=response)

In [7]:
producer.flush()